In [31]:
import pandas as pd

In [102]:
main_information = pd.read_csv("D:/DataSets/FinishedDatasets/MainInformation.csv")


In [33]:
user_id = list(range(0,100))

In [34]:
mock_user_data = pd.DataFrame()
mock_user_data["click_ID"] = None
mock_user_data["user_ID"] = None
mock_user_data["property_ID"] = None
mock_user_data["click_count"] = None

In [127]:
import pandas as pd
import numpy as np
from itertools import product

# Assuming user_id and main_information are defined
user_id = list(range(100))  # Replace this with your actual user IDs

# Generate all combinations of user IDs and property IDs
combinations = list(product(user_id, main_information["propertyID"]))

# Create DataFrame from combinations
mock_user_data = pd.DataFrame(combinations, columns=["user_ID", "property_ID"])

# Add click_id column indexed from 0
mock_user_data["click_id"] = range(len(mock_user_data))

# Simulate clicks with a normal distribution
mean_clicks = 50  # Adjust the mean as needed
std_dev_clicks = 60  # Adjust the standard deviation as needed


# Simulate clicks with a skewed distribution (gamma distribution)
shape_parameter = 2  # Adjust the shape parameter as needed
scale_parameter = 20  # Adjust the scale parameter as needed

# Generate random click counts from a gamma distribution
mock_user_data["click_count"] = np.random.gamma(shape_parameter, scale_parameter, size=len(mock_user_data))
mock_user_data["click_count"] = np.round(np.clip(mock_user_data["click_count"], 0, 100)).astype(int)


In [128]:
mock_user_data

,user_ID,property_ID,click_id,click_count
0,0,36428890,0,20
1,0,36428953,1,18
2,0,36429007,2,90
3,0,36429137,3,4
4,0,36429337,4,38
...,...,...,...,...
250895,99,2124108834,250895,11
250896,99,2124248020,250896,61
250897,99,2125094297,250897,45
250898,99,2130171237,250898,60


In [168]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_users(user_ID, data):
   
    # begin by creating a dataframe that contains unique property IDs
    user_data = data[data['user_ID'] == user_ID].pivot_table(index='property_ID', values='click_count', aggfunc='sum').reset_index()
    
    # do the same but now make the property ID as rows, the user ID as columns, and the click count as values
    other_users_data = data[data['user_ID'] != user_ID].pivot_table(index='property_ID', columns='user_ID', values='click_count', aggfunc='sum').fillna(0)
    
    # we need to fix the formatting so that input is compatable with the cosine similarity formula, input must be a 2d array
    other_users_data = other_users_data.T.values
    
    # get unique ids so that similar patterns can be matched accordingly
    user_ids = data[data['user_ID'] != user_ID]['user_ID'].unique()
        
    # now apply the formula, and store in new dataframe
    similarity = cosine_similarity([user_data['click_count']], other_users_data)[0]
    
    # convert the similar users array into a dataframe
    similar_users_df = pd.DataFrame({'user_ID': user_ids, 'similarity': similarity})
    
    # Sort by similarity in descending order
    similar_users_df = similar_users_df.sort_values(by='similarity', ascending=False)

    # Merge with original data to get click counts for each property
    result_df = pd.merge(similar_users_df, data, on='user_ID')

    # Aggregate click counts per property
    result_df = result_df.pivot_table(index=['user_ID', 'property_ID'], values='click_count', aggfunc='sum').reset_index()

    # Sort by aggregated click count in descending order
    result_df = result_df.sort_values(by='click_count', ascending=False)
 
    return result_df

In [171]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_users(user_ID, data):
   
    # begin by creating a dataframe that contains unique property IDs
    user_data = data[data['user_ID'] == user_ID].pivot_table(index='property_ID', values='click_count', aggfunc='sum').reset_index()

    # do the same but now make the property ID as rows, the user ID as columns, and the click count as values
    other_users_data = data[data['user_ID'] != user_ID].pivot_table(index='user_ID', columns='property_ID', values='click_count', aggfunc='sum').fillna(0)

    # we need to fix the formatting so that input is compatable with the cosine similarity formula, input must be a 2d array
    similarity = cosine_similarity([user_data['click_count']], other_users_data.values)[0]

    # convert the similar users array into a dataframe
    similar_users_df = pd.DataFrame({'user_ID': other_users_data.index, 'similarity': similarity})

    # now apply the formula, and store in new dataframe
    similar_users_df = similar_users_df.sort_values(by='similarity', ascending=False)

    # extract the top similar users we will find 10 most similar users
    top_similar_users = similar_users_df.head(10)  

    # merge datset, this way we can find the counts
    result_df = pd.merge(top_similar_users, data, on='user_ID')

    # aggregate counts
    result_df = result_df.pivot_table(index=['property_ID'], values='click_count', aggfunc='sum').reset_index()

    # sort by aggregated click count in descending order
    result_df = result_df.sort_values(by='click_count', ascending=False)
    
    result_df = result_df.head(10)
 
    return result_df

In [169]:
hi = find_similar_users(0, mock_user_data)

In [170]:
hi

,property_ID,click_count
1441,2065068090,745
2131,2085662193,697
477,2055486335,665
373,2055282471,648
729,2056554283,644
594,2055940693,639
1667,2071003451,630
1161,2059932598,626
1716,2072601833,621
2327,2092471675,619


In [139]:
hi.to_csv("D:/DataSets/FinishedDatasets/mock_user_data.csv", index = False)